### Tensorflow提供了通过变量名称来创建和获取一个变量的机制。通过这个机制，在不同的函数中可以通过变量的名字来调用变量，而不需要将变量通过参数的形式将变量到处传递

In [1]:
import tensorflow as tf

/home/ajialala/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# 下面这两个变量定义是等价的
v = tf.get_variable('v', shape=[1], initializer=tf.constant_initializer(1.0))
v = tf.Variable(tf.constant(1.0, shape=[1], name='v'))

![初始化函数](./pic/变量_1.png)

### tf.get_variable与tf.Variable最大的区别在于指定变量名称的参数。对于tf.Variable函数，变量名称是一个可选的参数。tf. Variable不管在什么情况下都是创建新变量，自己解决命名冲突。但是对于tf.get_variable函数，变量名是一个必填的参数。tf.get_variable会根据这个变量名去创建或者获取变量。

In [ ]:
# 在名字为‘foo’的命名空间内创建名字为v的变量。
with tf.variable_scope('foo'):
    v = tf.get_variable('v', [1], initializer=tf.constant_initializer(1.0))

In [6]:
# 因为在命名空间‘foo’中已经创建了名字为v的变量，所以下面的代码将会报错。
# ValueError: Variable foo/v already exists, disallowed. 
# Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:
with tf.variable_scope('foo'):
    v = tf.get_variable('v', [1])

ValueError: Variable foo/v already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "<ipython-input-3-d361701549e4>", line 3, in <module>
    v = tf.get_variable('v', [1], initializer=tf.constant_initializer(1.0))
  File "/home/ajialala/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2963, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/home/ajialala/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2903, in run_ast_nodes
    if self.run_code(code, result):


在生成上下文管理器时，将参数reuse设置为True，这样tf.get_variable将直接获取已经声明的变量。

In [10]:
with tf.variable_scope('foo', reuse=True):
    v1 = tf.get_variable('v', [1])
    print(v == v1)
    print(v)
    print(v1)
    print(v.name)

False
<tf.Variable 'foo/vvvv:0' shape=(1,) dtype=float32_ref>
<tf.Variable 'foo/v:0' shape=(1,) dtype=float32_ref>
foo/vvvv:0


variable_scope与get_variable搭配使用可以使得共享变量
name_scope主要用来tensorboard可视化